In [ ]:
!wget 'https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0' positive.csv
!wget 'https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0' negative.csv

In [1]:
import pandas as pd

In [2]:
positive = pd.read_csv('positive.csv?dl=0',header=None,encoding='utf-8',delimiter=';')
negative = pd.read_csv('negative.csv?dl=0',header=None,encoding='utf-8',delimiter=';')

In [3]:
positive=positive[[positive.columns[3]]]
negative=negative[[negative.columns[3]]]

In [4]:
positive['target']=1
negative['target']=0

In [5]:
positive.columns=['text','target']
negative.columns=['text','target']

In [6]:
positive.head()

,text,target
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [7]:
full_set = pd.concat([positive,negative])

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [9]:
x_train,x_test,y_train,y_test = train_test_split(full_set['text'],full_set.target.values,test_size=0.2,shuffle=True,random_state=2121)

In [10]:
clf = make_pipeline(TfidfVectorizer(ngram_range=(1,2)),MultinomialNB())
clf.fit(x_train,y_train)
preds = clf.predict(x_test)

In [11]:
accuracy_score(y_test,preds)

0.7739546366301496

## Теперь опробуем на этих же данных простейшую сеть с предобученными ембедингами

In [ ]:
!wget http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.vec

In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [13]:
f = open('ft_native_300_ru_twitter_nltk_word_tokenize.vec')
embedding_values = {}
for line in tqdm_notebook(f):
    value = line.replace('\n','').split(' ')
    word = value[0]
    coef = np.array(value[1:-1],dtype = 'float32')
    embedding_values[word]=coef

A Jupyter Widget

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(full_set['text'])
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 291376 unique tokens.


In [15]:
embedding_matrix = np.zeros((len(word_index)+1, 100),dtype=np.float32)
sum_finding=0
for word, i in tqdm_notebook(word_index.items()):
    try:
        embedding_vector = embedding_values[word]
        sum_finding=sum_finding+1
    except:
        embedding_vector = embedding_values["unknown"]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

A Jupyter Widget

In [16]:
print('Нашли векторов - ',sum_finding)

Нашли векторов -  147597


In [17]:
embedding_matrix.shape

(291377, 100)

In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337) 

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, SpatialDropout1D
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop,Adam

max_features = 20000
maxlen = 40  
batch_size = 256 

print('Готовим модель...')
model = Sequential()
model.add(Embedding(
    len(word_index)+1,
    100,
    weights=[embedding_matrix],
    input_length=maxlen,
    trainable=False
))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(32)))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-3,clipnorm=5,clipvalue=5),
              metrics=['accuracy'])

X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

print('Обучаем...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=25,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Тест score:', score)
print('Тест accuracy:', acc)